# 14. Тонкая настройка и контроль процесса обучения через метод fit() 

In [1]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.datasets import cifar10, mnist
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt

2024-12-01 22:03:35.834647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 22:03:35.982881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 22:03:36.025696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [4]:
model = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, batch_size=64, epochs=5)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8642 - loss: 0.4881
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - accuracy: 0.9618 - loss: 0.1244
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 851us/step - accuracy: 0.9765 - loss: 0.0792
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 933us/step - accuracy: 0.9824 - loss: 0.0571
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 913us/step - accuracy: 0.9872 - loss: 0.0429


In [9]:
#параметры валидации

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

model2 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(x_train_data, y_train_data, batch_size=64, epochs=5, validation_data=(x_train_val, y_train_val))

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8436 - loss: 0.5477 - val_accuracy: 0.9528 - val_loss: 0.1617
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9569 - loss: 0.1428 - val_accuracy: 0.9636 - val_loss: 0.1251
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9717 - loss: 0.0945 - val_accuracy: 0.9650 - val_loss: 0.1156
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9813 - loss: 0.0677 - val_accuracy: 0.9668 - val_loss: 0.1129
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.9858 - loss: 0.0506 - val_accuracy: 0.9675 - val_loss: 0.1119


In [11]:
#параметры валидации tf.data

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model3 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(train_dataset, epochs=5, validation_data=val_dataset)

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8362 - loss: 0.5782 - val_accuracy: 0.9479 - val_loss: 0.1741
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9550 - loss: 0.1540 - val_accuracy: 0.9622 - val_loss: 0.1248
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9722 - loss: 0.0980 - val_accuracy: 0.9643 - val_loss: 0.1174
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9773 - loss: 0.0716 - val_accuracy: 0.9732 - val_loss: 0.0912
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9838 - loss: 0.0538 - val_accuracy: 0.9738 - val_loss: 0.0916


In [12]:
#steps_per_epoch and validation_steps

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model4 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model4.fit(train_dataset, epochs=5,steps_per_epoch=100, validation_data=val_dataset)

Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6001 - loss: 1.3832 - val_accuracy: 0.8936 - val_loss: 0.3731
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9076 - loss: 0.3267 - val_accuracy: 0.9215 - val_loss: 0.2754
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9210 - loss: 0.2756 - val_accuracy: 0.9283 - val_loss: 0.2368
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9190 - loss: 0.2552 - val_accuracy: 0.9365 - val_loss: 0.2144
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9374 - loss: 0.2159 - val_accuracy: 0.9398 - val_loss: 0.1960


In [13]:
validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model5 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model5.fit(train_dataset, epochs=5, validation_data=val_dataset, validation_steps=5)

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8378 - loss: 0.5623 - val_accuracy: 0.9531 - val_loss: 0.1638
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 962us/step - accuracy: 0.9559 - loss: 0.1487 - val_accuracy: 0.9750 - val_loss: 0.1194
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 929us/step - accuracy: 0.9708 - loss: 0.0962 - val_accuracy: 0.9531 - val_loss: 0.1161
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9786 - loss: 0.0706 - val_accuracy: 0.9375 - val_loss: 0.2044
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - accuracy: 0.9828 - loss: 0.0538 - val_accuracy: 0.9719 - val_loss: 0.1002


In [15]:
#изменение весов для выходных значений

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model6 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model6.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

class_weight = {
    0: 1000.0,
    1: 1.0,
    2: 1.0,
    3: 1.0,
    4: 1.0,
    5: 1.0,
    6: 1.0,
    7: 1.0,
    8: 1.0,
    9: 1.0,
}

model6.fit(train_dataset, epochs=5, validation_data=val_dataset, class_weight=class_weight)

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3939 - loss: 10.6588 - val_accuracy: 0.7895 - val_loss: 0.7669
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7367 - loss: 1.2522 - val_accuracy: 0.8301 - val_loss: 0.6233
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8316 - loss: 0.7256 - val_accuracy: 0.8863 - val_loss: 0.4168
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8843 - loss: 0.4586 - val_accuracy: 0.9137 - val_loss: 0.3109
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9133 - loss: 0.3406 - val_accuracy: 0.9267 - val_loss: 0.2653


In [18]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

sample_weight = np.ones(shape=(len(x_train),))
sample_weight[y_train == 1] = 5.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model7 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model7.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model7.fit(x_train, y_train, epochs=5, sample_weight=sample_weight)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 706us/step - accuracy: 0.8645 - loss: 0.5022
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 678us/step - accuracy: 0.9648 - loss: 0.1260
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.9760 - loss: 0.0842
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 695us/step - accuracy: 0.9832 - loss: 0.0583
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 709us/step - accuracy: 0.9876 - loss: 0.0438


In [19]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

sample_weight = np.ones(shape=(len(x_train),))
sample_weight[y_train == 1] = 5.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model8 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model8.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model8.fit(x_train, y_train, epochs=3, validation_split=0.2)
print(history.history)

Epoch 1/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 896us/step - accuracy: 0.8575 - loss: 0.4859 - val_accuracy: 0.9596 - val_loss: 0.1375
Epoch 2/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - accuracy: 0.9611 - loss: 0.1257 - val_accuracy: 0.9718 - val_loss: 0.0975
Epoch 3/3
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 816us/step - accuracy: 0.9749 - loss: 0.0808 - val_accuracy: 0.9728 - val_loss: 0.0912
{'accuracy': [0.9190000295639038, 0.9648541808128357, 0.9771249890327454], 'loss': [0.27640825510025024, 0.11386938393115997, 0.07493723928928375], 'val_accuracy': [0.9595833420753479, 0.971750020980835, 0.9727500081062317], 'val_loss': [0.13748396933078766, 0.0974680483341217, 0.09118197858333588]}


In [23]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

sample_weight = np.ones(shape=(len(x_train),))
sample_weight[y_train == 1] = 5.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model9 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model9.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


class DigitsLimit(keras.utils.Sequence):
    def __init__(self, x, y, batch_size, max_len = -1):
        self.batch_size = batch_size
        self.x = x[:max_len]
        self.y = y[:max_len]
        
    def __len__(self):
        return int(np.ceil(self.x.shape[0] / self.batch_size))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        p = np.random.permutation(len(self.x))
        self.x = self.x[p]
        self.y = self.y[p]
        print(' on_epoch_end')

sequence = DigitsLimit(x_train, y_train, 64)

history = model9.fit(sequence, epochs=3, shuffle=False)
print(history.history)

Epoch 1/3
917/938 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.8476 - loss: 0.5333 on_epoch_end
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8492 - loss: 0.5278  
Epoch 2/3
882/938 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.9628 - loss: 0.1232 on_epoch_end
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 983us/step - accuracy: 0.9629 - loss: 0.1228
Epoch 3/3
919/938 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.9758 - loss: 0.0800 on_epoch_end
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9758 - loss: 0.0800  
{'accuracy': [0.9150652289390564, 0.9651827812194824, 0.9758829474449158], 'loss': [0.29755184054374695, 0.11533674597740173, 0.07932576537132263]}


In [ ]:
#callbacks обратный вызов

tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

sample_weight = np.ones(shape=(len(x_train),))
sample_weight[y_train == 1] = 5.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

validation_split = 0.2
validation_split_index = np.ceil(x_train.shape[0] * validation_split).astype('int32')

#обучающая валидации
x_train_val = x_train[:validation_split_index]
y_train_val = y_train[:validation_split_index]

#обучающая выборка
x_train_data = x_train[validation_split_index:]
y_train_data = y_train[validation_split_index:]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_train_val, y_train_val))
val_dataset = val_dataset.batch(64)

model9 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model9.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.5, patience=1, verbose=1)
    ]

# min_delta =0.01 and patience = 2

model9.fit(x_train, y_train, epochs=3, callbacks=callbacks)


Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 726us/step - accuracy: 0.8766 - loss: 0.4229
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/step - accuracy: 0.9649 - loss: 0.1142
Epoch 2: early stopping


In [26]:
#ModelCheckpoint


tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


model10 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model10.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.01, patience=2, verbose=1),
    keras.callbacks.ModelCheckpoint(filepath="mymodel_{epoch}.keras", save_best_only='True', monitor='loss', verbose=1 )
    ]

# min_delta =0.01 and patience = 2

model10.fit(x_train, y_train, epochs=3, callbacks=callbacks)

Epoch 1/3
1836/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.8714 - loss: 0.4305
Epoch 1: loss improved from inf to 0.24260, saving model to mymodel_1.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 699us/step - accuracy: 0.8726 - loss: 0.4265
Epoch 2/3
1864/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.9663 - loss: 0.1094
Epoch 2: loss improved from 0.24260 to 0.10128, saving model to mymodel_2.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.9663 - loss: 0.1093
Epoch 3/3
1819/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.9781 - loss: 0.0717
Epoch 3: loss improved from 0.10128 to 0.06842, saving model to mymodel_3.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.9782 - loss: 0.0716


In [29]:
model11 = keras.models.load_model('mymodel_3.keras')

In [30]:
print(model11.evaluate(x_test, y_test))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step - accuracy: 0.9687 - loss: 0.0924
[0.07845163345336914, 0.973800003528595]


In [ ]:
#свой класс для оценки функции потерь


tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


model12 = keras.Sequential([
    layers.Input(shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
    ])

model12.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []
        
    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get('loss'))
        
    def on_train_end(self, logs):
        print(self.per_batch_losses[:5])

callbacks = [CustomCallback(),]

model12.fit(x_train, y_train, epochs=3, callbacks=callbacks)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 712us/step - accuracy: 0.8777 - loss: 0.4238
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 793us/step - accuracy: 0.9652 - loss: 0.1134
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.9777 - loss: 0.0733
[2.374800682067871, 2.382244825363159, 2.3005878925323486, 2.2811496257781982, 2.25882625579834]
